# Information Retrieval 2/2

Dans cette partie, l'objectif est de mettre en application les meilleurs méthodes d'Information Retrieval pour votre outil de RAG.



Pour cela vous devrez :


1.   Déterminer les modèles les plus appropriés (s'appuyer sur la partie précédente)
2.   Pour les modèles d'embeddings, déterminer une base de donnée vectorielle appropriée
3.   Implémenter la classe si dessous.

Conseils :
* Le framework *langchain* permet de réaliser ces tâches assez simplement.
* Vous pouvez tester les méthodes sur les passages test de la partie précédente et sur les documents de la première séance.
* Testez plusieurs méthodes (différents paramètres, renvoie d'un ou plusieurs passages concaténés...)

In [2]:
!pip install langchain
import langchain as lc

class TextRetriever:
    def __init__(self, model_name="facebook/bart-base", embedding_path="paraphrase-distilbert-base"):
        """
        Initialise les modèles et la base de données vectorielle.

        Args:
            model_name (str): Nom du modèle de langage à utiliser (ex: "facebook/bart-base").
            embedding_path (str): Chemin du répertoire où la base de données vectorielle est stockée.
        """
        self.model = lc.load(model_name)
        self.embeddings = lc.load_embeddings(embedding_path)

    def store_embeddings(self, chunks, path="embeddings.pkl"):
        """
        Stocke les embeddings des chunks de texte dans une base de données vectorielle.

        Args:
            chunks (list of str): Liste de chunks de texte à stocker.
            path (str): Chemin du répertoire où la base de données sera stockée.
        """
        embeddings = self.embeddings.encode(chunks)
        lc.save_embeddings(embeddings, path)

    def load_embeddings(self, path="embeddings.pkl"):
        """
        Charge les embeddings depuis une base de données vectorielle.

        Args:
            path (str): Chemin du répertoire de la base de données.
        """
        self.embeddings = lc.load_embeddings(path)

    def get_best_chunks(self, query, top_k=10):
        """
        Recherche les meilleurs chunks correspondant à une requête.

        Args:
            query (str): Requête de recherche.
            top_k (int): Nombre de meilleurs chunks à retourner.

        Returns:
            list: Liste des meilleurs chunks correspondant à la requête.
        """
        encoded_query = self.embeddings.encode([query])
        scores = self.embeddings.cosine_similarity(encoded_query, self.embeddings.vectors)
        top_k_indices = scores.argsort(axis=1)[:,-top_k:]
        top_chunks = [self.embeddings.docs[i] for i in top_k_indices.flatten()]
        return top_chunks

    def rerank_chunks(self, query, chunks):
        """
        Retrie les chunks par pertinence

        Args:
            query (str): Requête de recherche.
            chunks (list of str): Liste des chunks à reclasser.

        Returns:
            list: liste triée des chunks par pertinence
        """
        encoded_query = self.embeddings.encode([query])
        scores = self.embeddings.cosine_similarity(encoded_query, self.embeddings.encode(chunks))
        sorted_chunks = [chunks[i] for i in scores.argsort(axis=0)[:-1]]
        return sorted_chunks

    def get_context(self, query):
        """
        Retourne un texte contenant les informations pertinentes pour la requête.

        Args:
            query (str): Requête de recherche.

        Returns:
            str: texte pertinent pour répondre
        """
        top_chunks = self.get_best_chunks(query)
        return " ".join(top_chunks)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


## Questions


*   Décrivez et motivez la méthode choisie (modèles utilisés, nombre de passages renvoyés...)
*   Comment adapter la solution en cas de base de données plus grande?
*   Quels sont les avantages à utiliser une base de données vectorielle pour stocker les embeddings?



